## Camera Calibration

<p style='direction:rtl; text-align: right'>
برای پیدا کردن ماتریس کالیبراسیون، ابتدا با تابع findChessboardCorners به ازای هر عکس مختصات دو بعدی گوشه‌های صفحه شطرنجی
را به دست می‌آوریم و با تابع cornerSubPix نیز آن‌را ریفاین می‌کنیم. سپس با مختصات سه بعدی آن‌ها که به بر اساس ساختار صفحه
تولید شده‌است، متناظر می‌کنیم. پس از آن، برای این نقاط متناظر شده، با تابع calibrateCamera یک ماتریس کالیبراسیون دوربین به دست می‌آوریم.
<br>
<br>
توجه کنید که مختصات سه بعدی نقاط را در فضا ثابت در نظر می‌گیریم و فرض می‌کنیم دوربین نسبت به صفحه جا به جایی داشته‌است.
پس این نقاط سه بعدی را روی صفحه Z=0 در نظر میگیریم و روی جدولی با مربع‌های ۲۲میلی‌متری قرار می‌دهیم. (گرچه طول ضلع مربع‌ها
اهمیت خاصی ندارد و در نهایت بی‌تاثیر می‌شود)
</p>

In [1]:
import cv2 as cv
import numpy as np
import glob

all_images = []
image_names = glob.glob('./data/hw2/checkerboard/*.jpg')
for image_name in sorted(image_names):
    all_images.append(cv.imread(image_name))


In [2]:
def get_camera_matrix(images):
    board_size = (6, 9)

    pts2d = []
    pts3d = np.zeros((len(images), board_size[0] * board_size[1], 3), np.float32)
    pts3d[:, :, :2] = np.mgrid[0:board_size[0], 0:board_size[1]].T.reshape(-1, 2) * 0.022
    pts3d = list(pts3d)

    for img in images:
        img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        ret, corners = cv.findChessboardCorners(img_gray, board_size,
                                                 cv.CALIB_CB_ADAPTIVE_THRESH +
                                                 cv.CALIB_CB_FAST_CHECK +
                                                 cv.CALIB_CB_NORMALIZE_IMAGE)
        corners2 = cv.cornerSubPix(img_gray, corners, (11, 11), (-1, -1),
                                    (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        pts2d.append(corners2)

    _, mtx, _, _, _ = cv.calibrateCamera(pts3d, pts2d, images[0].shape[:2][::-1], None, None)
    return mtx


mat1 = get_camera_matrix(all_images[0:10])
print(f'1: {mat1}')
mat2 = get_camera_matrix(all_images[5:15])
print(f'2: {mat2}')
mat3 = get_camera_matrix(all_images[10:20])
print(f'3: {mat3}')
mat4 = get_camera_matrix(all_images[0:20])
print(f'4: {mat4}')

1: [[2.93177481e+03 0.00000000e+00 9.11515601e+02]
 [0.00000000e+00 2.95269125e+03 5.51569293e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
2: [[3.00242310e+03 0.00000000e+00 8.84347162e+02]
 [0.00000000e+00 2.99848927e+03 5.36376647e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
3: [[3.04620462e+03 0.00000000e+00 7.18349258e+02]
 [0.00000000e+00 3.03488553e+03 5.48592497e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
4: [[2.98242948e+03 0.00000000e+00 8.34059633e+02]
 [0.00000000e+00 2.99043576e+03 5.14525378e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


<p style='direction:rtl; text-align: right'>
همانطور که در بالاتر مشاهده می‌کنید، ماتریس‌های به دست آمده در شرایط مختلف تقریبا یکسان است و خطای نسبی آن‌ها را در ادامه
می‌توانید مشاهده کنید.
<br>
خطای موجود می‌تواند به دلیل وجود دیستورشن‌هایی در تصویر برداری باشد که در مدل‌سازی ما جای‌ ندارد. به علاوه، خطای عددی نیز سهم
زیادی می‌تواند داشته باشد زیرا محاسبات با float32 انجام شده‌است.
</p>



In [3]:
mats = [mat1, mat2, mat3, mat4]
dists = np.zeros((len(mats), len(mats)))

for i, matA in enumerate(mats):
    for j, matB in enumerate(mats):
        dists[i, j] = np.linalg.norm(matA - matB) / np.linalg.norm(matA)

print(dists)

[[0.         0.02089866 0.05566838 0.02481692]
 [0.02055262 0.         0.04027795 0.01348922]
 [0.05441723 0.04003562 0.         0.03265868]
 [0.02458533 0.01358834 0.03309779 0.        ]]


<p style='direction:rtl; text-align: right'>
فاصله‌ کانونی دوربین را بر اساس ماتریس به دست آمده از همه عکس‌ها حساب می‌کنیم.
با فرض مربعی بودن پیکسل‌ها، درایه اول ماتریس کالیبراسیون را به عنوان فاصله کانونی در نظر می‌گیریم.
</p>

In [4]:
print(f'Focal Distance = {mat4[0, 0]:.2f}px')


Focal Distance = 2982.43px
